Este notebook mostra como realizar busca semântica utilizando o modelo [sentence-transformer-ult5-pt-small](https://huggingface.co/tgsc/sentence-transformer-ult5-pt-small)

# Instalação dos pacotes

In [1]:
!pip install datasets sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Download do dataset

In [2]:
!wget https://github.com/thacio/Few-Shot-Editais-TI/raw/main/editais_TI.xlsx

--2023-05-17 22:01:56--  https://github.com/thacio/Notebooks-sentence-transformers/raw/main/editais_TI.xlsx
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/thacio/Few-Shot-Editais-TI/main/editais_TI.xlsx [following]
--2023-05-17 22:01:57--  https://raw.githubusercontent.com/thacio/Few-Shot-Editais-TI/main/editais_TI.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 702524 (686K) [application/octet-stream]
Saving to: ‘editais_TI.xlsx’

editais_TI.xlsx     100%[===================>] 686.06K  --.-KB/s    in 0.04s   

2023-05-17 22:01:57 (15.2 MB/s) - ‘editais_TI.xlsx’ saved [702524/702524]



O dataset consiste em objetos de editais de licitações, classificados em 'Tecnologia de informação' e 'Outras compras', caso não sejam de TI.

Esta célula carrega o dataset em pandas e elimina as linhas repetidas

In [3]:
import pandas
import numpy

pandas.set_option('display.max_colwidth', None)

df = pandas.read_excel('editais_TI.xlsx')
df['CLASSE_TI'] = df['CLASSE_TI'].replace({0: 'Outras compras', 1: 'Tecnologia da Informação'})
df['DES_OBJETO'] = df['DES_OBJETO'].str.replace('Objeto: Pregão Eletrônico - ', '').str.strip()
df = df[~df['DES_OBJETO'].duplicated()] # Remove colunas com objeto duplicado

df.head()

,Unnamed: 0,COD_UASG,NUM_ANO_PREGAO,DES_OBJETO,CLASSE_TI
0,0,153038,12019,"O objeto da presente licitação é de empresa para realização de serviços técnicos especializados na área de tecnologia da informação, contemplando a implantação e execução continuada das atividades de Gerenciamento e Operação da Central de Serviços de TIC de 1º, 2º e 3º níveis, para atender às necessidades da Universidade Federal da Bahia, em seus diversos órgãos e unidades situados nas cidades de Salvador, Vitória da Conquista, Oliveira dos Campinhos (Santo Amaro) e Camaçari, conforme condições, quantidades e exigências estabelecidas neste Edital e seus anexos.",Outras compras
1,1,153054,122019,"Registro de Preços para futuras aquisições de material farmacológico (anastrozol, cisplatina, citarabina e outros) para o Hospital das Clínicas-UFG/EBSERH",Outras compras
2,2,153054,152019,Registro de preços para futuras aquisições de material farmacológico (carboplatina 150 mg e outros).,Outras compras
3,3,153054,192019,Contratação de empresa para prestação de serviço especializado de portaria com dedicação de mão de obra exclusiva.,Outras compras
4,4,153054,332019,Registro de preço para futuras aquisições de Órtese e Prótese (Cardiodesfibrilador com marca passo multi-sitio e outros).,Outras compras


# Cálculo dos embeddings

##Carrega o modelo

utilizando a biblioteca sentence transformers

In [4]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('tgsc/sentence-transformer-ult5-pt-small')

## Calcula os embeddings

dos objetos de licitação na coluna 'DES_OBJETO' e salva na coluna 'EMBEDDINGS'

In [5]:
# %%time
# df['EMBEDDINGS'] = df['DES_OBJETO'].apply(lambda x: model.encode(x,convert_to_tensor=True).cpu())

faz o mesmo procedimento, porém usando batchs para acelerar caso se utilize gpu

In [6]:
%%time
sentences = df['DES_OBJETO'].tolist()
batch_size = 512  # Adjust batch size as per your requirements
batches = [sentences[i:i+batch_size] for i in range(0, len(sentences), batch_size)]
embeddings = []

for batch in batches:
    batch_encodings = model.encode(batch,convert_to_tensor=True).cpu()
    embeddings.extend(batch_encodings)

df['EMBEDDINGS'] = embeddings

CPU times: user 11.6 s, sys: 1.34 s, total: 12.9 s
Wall time: 14.2 s


# Realiza a busca semântica.

1. Define o texto que será pesquisado
2. Calcula os embeddings do texto de pesquisa
3. Realiza o cálculo da similaridade por cosseno do texto a ser pesquisado com cada item do dataset
4. Ordena em ordem decrescente (quanto maior a similaridade por cosseno, mais próximo é a pesquisa)

## Busca por área de TI

texto: 'Compras relacionadas à area de Tecnologia da informação, softwares e materiais de informática'

In [7]:
%%time
from sentence_transformers import util

texto_pesquisa='Compras relacionadas à área de Tecnologia da informação, softwares e materiais de informática'

#Compute embedding for both lists
embeddings_pesquisa = model.encode(texto_pesquisa, convert_to_tensor=True)

# df = df.drop('SIMILARIDADE_COS', axis=1)
df['SIMILARIDADE_COS']=0.0
for idx, row in df.iterrows():
    #Compute cosine-similarities
    cosine_scores = util.cos_sim(embeddings_pesquisa.cpu(), row['EMBEDDINGS'])
    df.at[idx, 'SIMILARIDADE_COS'] = cosine_scores[0].item()

CPU times: user 1.98 s, sys: 5.9 ms, total: 1.98 s
Wall time: 2.01 s


In [8]:
df_salvo = df.drop('EMBEDDINGS', axis=1).sort_values(by='SIMILARIDADE_COS', ascending=False)
df_salvo.to_excel('TI.xlsx')

Vamos contar qual a porcentagem dos 100 primeiros itens estão classificados como Tecnologia da informação

In [9]:
subset = df_salvo[0:100]
count = len(subset[subset['CLASSE_TI'] == 'Tecnologia da Informação'])  # Count occurrences of 'Tecnologia da informação'

percentage = (count / 100) * 100  # Calculate the percentage

print("Count:", count)
print("Percentage:", percentage, "%")

Count: 95
Percentage: 95.0 %


94% dos 100 primeiros elementos estão classificados como TI.
Vamos dar uma olhada nos 6 que estão classificados como 'Outras compras'

In [10]:
subset[subset['CLASSE_TI'] == 'Outras compras']['DES_OBJETO'].tolist()

['Aquisição de componentes eletrônicos',
 'Prestação de serviços técnicos comuns de telecomunicações e eletrônica de infraestrutura de dados.',
 'Servidores Especializados em Computação Cognitiva',
 'Aquisição de Macas Hidráulicas e Camas Eletrônicas.',
 'Contratação de empresa para de prestação de serviços que utiliza tecnologia de Gestão e Gerenciamento por sistema informatizado e integrado via web, em tempo real ou cartão magnético permitindo a transmissão de dados e movimentação diária por software via internet para manutenção preventiva e corretiva de veículos automotores.']

Os 20 melhores resultados

In [11]:
df_salvo.drop(['Unnamed: 0', 'COD_UASG', 'NUM_ANO_PREGAO'], axis=1).head(20)

,DES_OBJETO,CLASSE_TI,SIMILARIDADE_COS
1539,Aquisição de materiais de informática.,Tecnologia da Informação,0.805185
3010,Aquisição de equipamentos de tecnologia da informação (tic),Tecnologia da Informação,0.804059
3761,Aquisição de material de áudio e informática.,Tecnologia da Informação,0.793519
2562,"Aquisição de computadores, outros equipamentos de informática, bens permanentes e bens de consumo de informática",Tecnologia da Informação,0.789788
757,Aquisição de equipamentos de Tecnologia da Informação.,Tecnologia da Informação,0.789361
6515,"LOTE I - Prestação de serviços técnicos especializados na Área de Tecnologia da Informação contemplando desenvolvimento, manutenção e documentação de Sistemas de Informação, a serem desenvolvidos sob a modalidade de Fábrica de Software. LOTE II - Prestação de serviços especializados de medição de sistemas em desenvolvimento, sistemas em produção, manutenção de sistema em produção, coleta de dados, geração e análise de indicadores e consultoria na aplicação de técnicas.",Tecnologia da Informação,0.784095
3980,"Registro de preços para aquisição de bens de informática (Salas Multimídias) para reuniões, brifing e instrução. Solução de sistemas de projeção multimídia, sistemas de monitores informativas e equipamentos de apoio para atender as necessidades do GUEs-9ª Bda Inf Mtz.",Tecnologia da Informação,0.776796
280,Aquisição de equipamentos de informática - SRP,Tecnologia da Informação,0.763633
4112,"Aquisição de equipamentos de comunicação de dados, gerenciamento e segurança da informação, armazenamento de dados e imagens e equipamentos de telecomunicações para conexão entre sites com treinamento e instalação, conforme condições, quantidades, exigências e estimativas estabelecidas no Termo de Referência e demais anexos.",Tecnologia da Informação,0.760716
1537,Aquisição de Equipamentos de Informática.,Tecnologia da Informação,0.756132


Os 10 últimos resultados

In [12]:
df_salvo.drop(['Unnamed: 0', 'COD_UASG', 'NUM_ANO_PREGAO'], axis=1).tail(10)

,DES_OBJETO,CLASSE_TI,SIMILARIDADE_COS
5673,"Execução de serviços de desobstrução do leito através de espigões/guias corrente no Rio Pindaré (Km 294), no município de Alto Alegre do Pindaré, na jurisdição da Bacia do Nordeste.",Outras compras,0.091719
219,"Prestação de serviços continuados de bombeiro civil (brigada de incêndio) na área de segurança contra incêndio, pânico, abandono de edificações, primeiros socorros e desenvolvimento e implantação de política prevencionista de segurança contra incêndio para atuação nas Edificações da Câmara dos Deputados, pelo período de doze meses.",Outras compras,0.085297
5251,"Aquisição de medicamentos, para o período de 12 (doze) meses.",Outras compras,0.080602
5017,Prestação de serviços continuados de vigilância armada e desarmada,Outras compras,0.071441
6159,Medalha Militar com Estojo e Medalha Mérito Marinheiro,Outras compras,0.062194
3510,Aquisição de uniformes especiais a fim de atender necessidade do Batalhão da Guarda Presidencial.,Outras compras,0.052208
210,"Prestação de serviços continuados de operação de elevadores, pelo período de doze meses.",Outras compras,0.051932
6464,Reparo nas comportas de vertedouro da barragem da uhe funil,Outras compras,0.048317
988,Endovascular.,Outras compras,0.036822
1243,Hortifrutigranjeiros,Outras compras,0.034227


## Busca por área de Educação 

texto: 'Compras relacionadas à área de educação, escolas e ensino.'

Agora vamos pesquisar compras relacionadas à área de educação

In [13]:
%%time
from sentence_transformers import util

texto_pesquisa='Compras relacionadas à área de educação, escolas e ensino.'

#Compute embedding for both lists
embeddings_pesquisa = model.encode(texto_pesquisa, convert_to_tensor=True)

# df = df.drop('SIMILARIDADE_COS', axis=1)
df['SIMILARIDADE_COS']=0.0
for idx, row in df.iterrows():
    #Compute cosine-similarities
    cosine_scores = util.cos_sim(embeddings_pesquisa.cpu(), row['EMBEDDINGS'])
    df.at[idx, 'SIMILARIDADE_COS'] = cosine_scores[0].item()

df_salvo = df.drop('EMBEDDINGS', axis=1).sort_values(by='SIMILARIDADE_COS', ascending=False)
df_salvo.to_excel('educação.xlsx')

CPU times: user 3.12 s, sys: 44.2 ms, total: 3.17 s
Wall time: 3.21 s


Agora vamos ver quanto dos 100 primeiros estão classificados como 'Tecnologia da informação', e se estão, quais são os objetos

In [14]:
subset = df_salvo[0:100]
count = len(subset[subset['CLASSE_TI'] == 'Tecnologia da Informação'])  # Count occurrences of 'Tecnologia da informação'

percentage = (count / 100) * 100  # Calculate the percentage

print("Count:", count)
print("Percentage:", percentage, "%")

Count: 8
Percentage: 8.0 %


In [15]:
subset[subset['CLASSE_TI'] == 'Tecnologia da Informação']['DES_OBJETO'].tolist()

['Registro de preços para aquisição de conjuntos de robótica educacional destinados às escolas públicas estaduais, municipais e do Distrito Federal.',
 'Registro de preços para fornecimento equipamentos de tecnologia da informação em atendimento às demandas do campus sede e campi do interior do estado do Pará pertencentes a Universidade Federal Rural da Amazônia-UFRA, conforme condições, quantidades, exigências e estimativas estabelecidas neste edital e seus anexos.',
 'Aquisição de MATERIAIS PARA LOGÍSTICA E ESTRUTURAÇÃO DE ATIVIDADES DE ENSINO, PESQUISA, EXTENSÃO E INTERNACIONALIZAÇÃO, DENTRE OUTROS, ESPECIFICAMENTE PARA OS RESPECTIVOS PLANOS DE COMUNICAÇÃO DE EVENTOS ACADÊMICOS, CIENTÍFICOS, ADMINISTRATIVOS E MISSÕES OFICIAIS por esta SCS/UFPI e seus quatro órgãos vinculados que atendem a TODOS OS CAMPI DA UNIVERSIDADE FEDERAL DO PIAUÍ, em atividades de ensino, pesquisa, extensão, internacionalização e missões oficiais',
 'Aquisição de material de consumo (Suprimentos de Informática

Tivemos três que não teriam relação com a área de educação, sendo os demais pertinentes.

---

Os 10 melhores resultados

In [16]:
df_salvo.drop(['Unnamed: 0', 'COD_UASG', 'NUM_ANO_PREGAO'], axis=1).head(10)

,DES_OBJETO,CLASSE_TI,SIMILARIDADE_COS
2280,"Registro de preços para aquisição de conjuntos de robótica educacional destinados às escolas públicas estaduais, municipais e do Distrito Federal.",Tecnologia da Informação,0.736919
3218,"O presente Pregão Eletrônico tem por objeto a AQUISIÇÃO PARCELADA E FUTURA DE PÃO BRIOCHE E OUTROS, necessários para o preparo da merenda escolar dos alunos do Campus Campos-Centro.",Outras compras,0.661964
2285,"Registro de preços com vistas a aquisição de bicicleta e capacete escolar, em atendimento às entidades educacionais das redes públicas de ensino nos Estados, Distrito Federal e Municípios.",Outras compras,0.661378
3186,Eventual aquisição de merenda escolar industrializada para os discentes do IFCE Campus Tianguá e demais órgãos participantes.,Outras compras,0.651609
3250,Aquisição de gêneros alimentícios destinados ao preparo e distribuição da alimentação escolar para os alunos do Campus Pau dos Ferros do IFRN e participantes.,Outras compras,0.641125
3220,"Aquisição parcelada e futura de ACHOCOLATADO E OUTROS, pelo SISTEMA DE REGISTRO DE PREÇOS (SRP) necessários para o preparo da merenda escolar dos alunos do Campus Campos-Centro do INSTITUTO FEDERAL FLUMINENSE.",Outras compras,0.641014
2593,Eventual aquisição de bens de consumo e permanentes de TI e eletrônica para o Campus Poços de Caldas - IFSULDEMINAS.,Outras compras,0.621015
3076,Aquisição de Merenda Escolar para o Campus Canoas e Demais Participantes.,Outras compras,0.620139
4324,"Contratação de pessoa jurídica especializada na prestação de serviços de AGENTE DE INTEGRAÇÃO DE ESTÁGIO (INTERMEDIAÇÃO PARA SELEÇÃO, CONTRATAÇÃO E ACOMPANHAMENTO DE ESTAGIÁRIOS), visando atender aos estudantes de educação superior e de ensino médio, vinculados à estrutura do ensino público e privado do país, para preenchimento de oportunidades de estágio no âmbito da SAMF/PB e de órgãos por ela atendidos, conforme as especificações técnicas e condições constantes do Edital e seus Anexos.",Outras compras,0.619120
1772,Produção de Materiais Gráficos para atendimento da demanda de produção de Diversos Livros Didáticos e Acadêmicos para atender as diversas Unidades Administrativas e Acadêmicas da UFES.,Outras compras,0.603932


Os 10 piores

In [17]:
df_salvo.drop(['Unnamed: 0', 'COD_UASG', 'NUM_ANO_PREGAO'], axis=1).tail(10)

,DES_OBJETO,CLASSE_TI,SIMILARIDADE_COS
5301,"Aquisição de mosquiteiros impregnados com inseticida, tipo piretróide, para rede, e mosquiteiros impregnados com inseticida, tipo piretróide, para cama, cônico",Outras compras,0.022758
5233,Alumínio duro - sem impressão com primer - 344mm - 20 micra.,Outras compras,0.022067
4815,Contratação de serviço de manutenção de equipamentos portáteis para combate à incêndio.,Outras compras,0.015927
4746,"Contratação de empresa especializada na prestação de serviços ininterruptos de segurança contra incêndio, pânico, abandono de edificação e primeiros socorros por meio de grupo de brigada de incêndio, executando ações preventivas e de emergência contra incêndio e outras ocorrências, com fornecimento de mão de obra e equipamentos necessários a execução do serviço para atender às necessidades da Procuradoria Geral da República- PGR, no Edifício-Sede e seus Anexos.",Outras compras,0.014281
5917,Contratação de empresa especializada na prestação dos serviços continuados de manutenção do sistema viário do Porto de Itaguaí.,Outras compras,0.014274
759,"Aquisição de embarcação inflável, flutuador salva-vidas e colete tático de busca e salvamento para o Corpo de Bombeiro Militar do Estado do Rio de Janeiro/CBMERJ",Outras compras,0.011259
4918,Bolsa coletora para nefrostomia,Outras compras,0.010200
4883,Minociclina 100mg,Outras compras,0.007206
795,"Contratação de empresa prestadora de serviços de segurança contra incêndio, pânico, abandono de edificação, elaboração e execução de Plano de Prevenção de Combate a Incêndio e primeiros socorros, por meio de Brigada de Bombeiro Civil , com o fornecimento dos respectivos equipamentos de proteção individual, equipamentos de proteção coletiva e de material de primeiros socorros, em Brasília DF",Outras compras,0.007113
4161,Furadeiras de furação profunda.,Outras compras,-0.031168


## Busca por área de Saúde

texto: 'Compras relacionadas à área de saúde, hospitais e remédios.'

In [18]:
%%time
from sentence_transformers import util

texto_pesquisa='Compras relacionadas à área de saúde, hospitais e remédios.'

#Compute embedding for both lists
embeddings_pesquisa = model.encode(texto_pesquisa, convert_to_tensor=True)

df['SIMILARIDADE_COS']=0.0
for idx, row in df.iterrows():
    #Compute cosine-similarities
    cosine_scores = util.cos_sim(embeddings_pesquisa.cpu(), row['EMBEDDINGS'])
    df.at[idx, 'SIMILARIDADE_COS'] = cosine_scores[0].item()

df_salvo = df.drop('EMBEDDINGS', axis=1).sort_values(by='SIMILARIDADE_COS', ascending=False)
df_salvo.to_excel('saúde.xlsx')

CPU times: user 3.99 s, sys: 57.5 ms, total: 4.04 s
Wall time: 4.17 s


Agora vamos ver quanto dos 100 primeiros estão classificados como 'Tecnologia da informação', e se estão, quais são os objetos

In [19]:
subset = df_salvo[0:100]
count = len(subset[subset['CLASSE_TI'] == 'Tecnologia da Informação'])  # Count occurrences of 'Tecnologia da informação'

percentage = (count / 100) * 100  # Calculate the percentage

print("Count:", count)
print("Percentage:", percentage, "%")

Count: 0
Percentage: 0.0 %


In [20]:
subset[subset['CLASSE_TI'] == 'Tecnologia da Informação']['DES_OBJETO'].tolist()

[]

Os 10 melhores resultados

In [21]:
df_salvo.drop(['Unnamed: 0', 'COD_UASG', 'NUM_ANO_PREGAO'], axis=1).head(10)

,DES_OBJETO,CLASSE_TI,SIMILARIDADE_COS
2086,"Registro de preços de produtos para saúde para atender ao Serviço de Cirurgia Vascular - Extensores, conjuntos introdutores, cateteres, fios guias, endopróteses, filtros, espirais aneurismas, patch´s, micropartículas, enxertos, selante cirúrgico e cola biológica.",Outras compras,0.800616
2831,"Aquisição parcelada de medicamentos oncológicos, soluções parenterais de grande volume, medicamento de uso odontológico, medicamentos especialmente manipulados e outros medicamentos de uso geral para utilização nos pacientes submetidos a procedimentos ambulatórios, clínicos, cirúrgicos e/ou ortopédicos no âmbito do HUMAP - UFMS",Outras compras,0.797028
2687,Produtos para Saúde,Outras compras,0.787512
2074,"Aquisição de produtos para atender o serviço de farmácia: anestésicos, relaxantes musculares e controlados.",Outras compras,0.781872
2068,Aquisição de produtos para atender o serviço de patologia clínica - Reagente para diagnóstico clínico e outros ( Com cessão de equipamento),Outras compras,0.780540
2981,Aquisição de produtos para saúde.,Outras compras,0.780003
2945,Produtos para Saúde ( Cirurgia Cardiaca),Outras compras,0.779868
3749,Aquisição de material de consumo hospitalar - curativos,Outras compras,0.770125
3204,"Aquisição de Material de Consumo Médico Hospitalar - (Equipos, Extensores, etc.)",Outras compras,0.763384
2064,"Produtos para saúde para atender o serviço de cirurgia vascular - extensores, conjuntos introdutores, cateteres, fios guias, endopróteses, filtros, espirais aneurismas, patch´s, micropartículas, enxertos, selante cirúrgico e cola biológica.",Outras compras,0.761145


Os 10 piores resultados

In [22]:
df_salvo.drop(['Unnamed: 0', 'COD_UASG', 'NUM_ANO_PREGAO'], axis=1).tail(10)

,DES_OBJETO,CLASSE_TI,SIMILARIDADE_COS
759,"Aquisição de embarcação inflável, flutuador salva-vidas e colete tático de busca e salvamento para o Corpo de Bombeiro Militar do Estado do Rio de Janeiro/CBMERJ",Outras compras,0.092000
4028,"Aquisição de Caminhões Especializados para o 7º Batalhão de Engenharia de Combate, localizado na cidade de Natal-RN; Comando do 1º Grupamento de Engenharia, localizado em João Pessoa-PB; 1º Batalhão de Engenharia de Construção, localizado em Caicó-RN; 2º Batalhão de Engenharia de Construção, localizado em Teresina-PI, 3º Batalhão de Engenharia de Construção, localizado em Picos-PI; e 4º Batalhão de Engenharia de Construção, localizado em Barreiras-BA conforme edital.",Outras compras,0.088345
3622,"Aquisição de Blocos de concreto de 16 (dezesseis faces) para a implantação da infraestrutura da 22ª Brigada da Foz localizada em Macapá-AP, obra realizada pela Operação Brigada da Foz 8º Batalhão de Engenharia de Construção",Outras compras,0.083608
42,Aquisição de Bobina de Gelo,Outras compras,0.081944
3962,Aquisição de instrumentos musicais para o 63º Batalhão de Infantaria.,Outras compras,0.080407
5233,Alumínio duro - sem impressão com primer - 344mm - 20 micra.,Outras compras,0.075540
3100,Elaboração de projetos de Engenharia.,Outras compras,0.070496
3627,Contratação de empresa para extração de areia,Outras compras,0.063708
6594,Transformador de excitacao,Outras compras,0.062470
6487,"Cabo isolado 500 mm classe de tensão 8,7 a 15KV.",Outras compras,0.015492


## Busca por área de Construção civil

texto: 'Compras relacionadas à área de construção civil, obras e rodovias.'

In [23]:
%%time
from sentence_transformers import util

texto_pesquisa='Compras relacionadas à área de construção civil, obras e rodovias.'

#Compute embedding for both lists
embeddings_pesquisa = model.encode(texto_pesquisa, convert_to_tensor=True)

# df = df.drop('SIMILARIDADE_COS', axis=1)
df['SIMILARIDADE_COS']=0.0
for idx, row in df.iterrows():
    #Compute cosine-similarities
    cosine_scores = util.cos_sim(embeddings_pesquisa.cpu(), row['EMBEDDINGS'])
    df.at[idx, 'SIMILARIDADE_COS'] = cosine_scores[0].item()

df_salvo = df.drop('EMBEDDINGS', axis=1).sort_values(by='SIMILARIDADE_COS', ascending=False)
df_salvo.to_excel('construção_civil.xlsx')

CPU times: user 3.17 s, sys: 15.1 ms, total: 3.19 s
Wall time: 3.2 s


Agora vamos ver quanto dos 100 primeiros estão classificados como 'Tecnologia da informação', e se estão, quais são os objetos

In [24]:
subset = df_salvo[0:100]
count = len(subset[subset['CLASSE_TI'] == 'Tecnologia da Informação'])  # Count occurrences of 'Tecnologia da informação'

percentage = (count / 100) * 100  # Calculate the percentage

print("Count:", count)
print("Percentage:", percentage, "%")

Count: 1
Percentage: 1.0 %


In [25]:
subset[subset['CLASSE_TI'] == 'Tecnologia da Informação']['DES_OBJETO'].tolist()

['Aquisição de Material Permanente de Infraestrutura.']

Os 10 melhores resultados

In [26]:
df_salvo.drop(['Unnamed: 0', 'COD_UASG', 'NUM_ANO_PREGAO'], axis=1).head(10)

,DES_OBJETO,CLASSE_TI,SIMILARIDADE_COS
3255,Aquisição de Material de Construção Civil.,Outras compras,0.751227
5453,"Contratação de empresa especializada para prestação de serviços de transporte rodoviário estadual e interestadual de mobílias, bagagens e volumes em geral, abrangendo todo território nacional, conforme especificações técnicas constantes no Edital e Termo de referência.",Outras compras,0.746193
4724,"Registro de preços para eventual contratação de serviços nas áreas de comércio exterior e logística, compreendendo agenciamento de carga internacional, despacho aduaneiro, transporte rodoviário e transporte aéreo nacional.",Outras compras,0.720587
868,Contratação de serviços comuns de engenharia de manutenção e conservação de bens imóveis.,Outras compras,0.720432
5007,"Prestação de serviços de transporte rodoviários de carga, local e interestadual, na modalidade de porta a porta, compreendendo bens patrimoniais de interesse do Ministério da Ciência, Tecnologia, Inovações e Comunicações, conforme condições, quantidades e exigências estabelecidas no Edital e seus anexos.",Outras compras,0.707562
4622,"Eventual contratação de empresa de engenharia especializada na prestação serviços, sob demanda, de manutenção predial com fornecimento de materiais, equipamentos e mão de obra, na forma estabelecida nas planilhas de serviços e insumos diversos descritos no Sistema Nacional de Pesquisa de Custos e Índices da Construção Civil, doravante denominado SINAPI, nas áreas, instalações e infraestruturas das unidades da Policia Rodoviária Federal em Roraima.",Outras compras,0.701960
6240,"Registro de preços para eventual contratação de serviços especializados nas áreas de metalurgia, montagem industrial, mecânica, eletricidade, mecânica naval, usinagem, fabricação de estruturas, peças e tubulações metálicas, conforme condições, quantidades e exigências estabelecidas neste Edital e seus anexos.",Outras compras,0.699701
6658,"Contratação, sob regime de empreitada por preço unitário, de empresa especializada na prestação dos serviços de limpeza de faixas de servidão (em trechos contínuos e/ou não contínuos), estradas de acesso, aceiros e corte de árvores isoladas, erradicação de vegetação, poda seletiva de vegetação, instalação e recuperação de porteiras, colchetes e cercas, aterramento e seccionamento de cercas em linhas de transmissão, localizadas em diversos municípios dos estados de Minas Gerais e São Paulo, nas áreas sob responsabilidade da Gerência de Produção Minas GRM.O, conforme condições estabelecidas no Edital, bem como o Código de Ética e de Conduta e seu Programa de Compliance .",Outras compras,0.698780
2549,Eventual aquisição de materiais para construção.,Outras compras,0.697559
3598,Aquisição de material de construção,Outras compras,0.694994


Os 10 piores resultados

In [27]:
df_salvo.drop(['Unnamed: 0', 'COD_UASG', 'NUM_ANO_PREGAO'], axis=1).tail(10)

,DES_OBJETO,CLASSE_TI,SIMILARIDADE_COS
5239,"Cloroquina, difosfato",Outras compras,0.108113
5185,Aquisição de Toalhinha Wiper,Outras compras,0.098131
6487,"Cabo isolado 500 mm classe de tensão 8,7 a 15KV.",Outras compras,0.095127
3466,Aquisição de alimentos para animais do zoológico do CIGS,Outras compras,0.093934
3438,Mamógrafo,Outras compras,0.092784
5535,"Aquisição de aquecedor leite humano sem agua, espectrofotometro, dispensador de parafina e outros",Outras compras,0.084479
5241,"Sílica gel em sache de 100g , como indicador de umidade acondicionado, individualmente, a vácuo em saco plástico.",Outras compras,0.082011
42,Aquisição de Bobina de Gelo,Outras compras,0.078476
4918,Bolsa coletora para nefrostomia,Outras compras,0.039909
5522,"Aquisição de leite em pó desnatado instantâneo, formula em pó a base de proteína, farinha de trigo e outros.",Outras compras,0.001479


## Busca por área indígena

texto: 'Licitações relacionadas à grupos indígenas'

In [28]:
%%time
from sentence_transformers import util

texto_pesquisa='Compras relacionadas à comunidades indígenas'

#Compute embedding for both lists
embeddings_pesquisa = model.encode(texto_pesquisa, convert_to_tensor=True)

# df = df.drop('SIMILARIDADE_COS', axis=1)
df['SIMILARIDADE_COS']=0.0
for idx, row in df.iterrows():
    #Compute cosine-similarities
    cosine_scores = util.cos_sim(embeddings_pesquisa.cpu(), row['EMBEDDINGS'])
    df.at[idx, 'SIMILARIDADE_COS'] = cosine_scores[0].item()

df_salvo = df.drop('EMBEDDINGS', axis=1).sort_values(by='SIMILARIDADE_COS', ascending=False)
df_salvo.to_excel('indígena.xlsx')

CPU times: user 3.21 s, sys: 15.9 ms, total: 3.23 s
Wall time: 3.23 s


Agora vamos ver quanto dos 100 primeiros estão classificados como 'Tecnologia da informação', e se estão, quais são os objetos

In [29]:
subset = df_salvo[0:100]
count = len(subset[subset['CLASSE_TI'] == 'Tecnologia da Informação'])  # Count occurrences of 'Tecnologia da informação'

percentage = (count / 100) * 100  # Calculate the percentage

print("Count:", count)
print("Percentage:", percentage, "%")

Count: 0
Percentage: 0.0 %


Os 10 melhores resultados

In [30]:
df_salvo.drop(['Unnamed: 0', 'COD_UASG', 'NUM_ANO_PREGAO'], axis=1).head(10)

,DES_OBJETO,CLASSE_TI,SIMILARIDADE_COS
4414,"Aquisição de gêneros alimentícios e materiais diversos destinados aos indígenas das diversas etnias jurisdicionados à Coordenação Regional do Maranhão, às Coordenações Técnicas Locais subordinadas a esta e à Frente de Proteção Etnoambiental-FPEA, que residem nas Terras Indígenas dos municípios de Arame, Amarante, Montes Altos, Grajaú, Jenipapo dos Vieiras, Barra do Corda, Santa Inês, Zé Doca e São Luís.",Outras compras,0.725215
4569,"Registro de preços para a aquisição eventual e futura de ferramentas e insumos agrícolas, de equipamentos para a produção de farinha-de-mandioca, de material de pesca e de equipamentos para geração de energia solar, conforme especificações e quantidades constantes neste Termo de Referência, para apoiarem e fomentarem as atividades produtivas e extrativistas tradicionais das comunidades indígenas jurisdicionadas à Coordenação Regional Kayapó Sul do Pará.",Outras compras,0.657313
5578,"Aquisição e fornecimento de hipoclorito de sódio, concentração mínima de cloro ativo de 12%, para utilização em água destinada ao consumo humano nas comunidades indígenas abrangidas pelo Distrito Sanitário Especial Indígena de Mato Grosso do Sul (DSEI-MS).",Outras compras,0.635683
5435,"Contratação de empresa especializada no fornecimento com instalação de Rádios Transceptores, para dar suporte às ações de comunicação nas comunidades Indígenas atendidas pelo Distrito Sanitário Especial Indígena Leste de Roraima.",Outras compras,0.620556
4574,"Aquisição de gêneros alimentícios com objetivo de suprir a demanda desta Coordenação Regional e Coordenações Técnicas Locais jurisdicionadas, para promover e/ou apoiar a realização de reuniões, seminários, conferências, oficinas, ações de fiscalização e monitoramento ambiental e territorial nas Terras Indígenas, e demais ações a serem desenvolvidas ou apoiadas pela FUNAI, no âmbito da Coordenação Regional Noroeste do Mato Grosso.",Outras compras,0.616110
4562,"Contratação de empresa(s) especializada(s) no fornecimento de Gêneros Alimentícios e Outros Produtos, de forma parcelada e de acordo com as demandas de projetos e ações, destinados a atender a Coordenação Regional , Coordenações Técnicas Locais e as ações da Comunidade Indígena Avá Canoeiro.",Outras compras,0.611896
5421,"Contratação de empresa especializada em serviços de emissão e fornecimento de bilhetes de passagens fluviais para atender pacientes indígenas e seus acompanhantes, servidores (a serviço), colaboradores (inclusive colaboradores eventuais a serviço) e conselheiros indígenas, na área de abrangência do DSEI Parintins, conforme condições, quantidades e exigências estabelecidas neste Edital e seus anexos.",Outras compras,0.603496
5580,"Contratação de empresa para prestação de serviço de preparo e manipulação de refeições nas pendências da Casa de Apoio a Saúde Indígena CASAI de JUARA/MT, COLÍDER/MT E PEIXOTO DE AZAVEDO/MT, para pacientes e acompanhantes indígenas pelas CASAI s jurisdicionadas ao DSEI Kayapó/MT.",Outras compras,0.596399
3640,Aquisição de Gêneros Alimentícios do Quantitativo de Rancho - HORTIFRUTIGRANJEIROS,Outras compras,0.594298
152,"Aquisição de Gêneros Alimentícios tal como, frios, polpas de frutas e laticínios, para atendimento das necessidades do Grupamento de Apoio de Belém (GAP-BE) e das Unidades Apoiadas.",Outras compras,0.590815


Os 10 piores resultados

In [31]:
df_salvo.drop(['Unnamed: 0', 'COD_UASG', 'NUM_ANO_PREGAO'], axis=1).tail(10)

,DES_OBJETO,CLASSE_TI,SIMILARIDADE_COS
810,Aquisição de pneus,Outras compras,-0.012805
399,Contratação de empresa especializada para a execução de serviços técnicos de substituição dos elevadores do edifício-sede do Superior Tribunal Militar.,Outras compras,-0.013062
837,"Serviço de reparo, reforma e/ou recuperação de componentes trabalháveis dos interiores de aeronaves.",Outras compras,-0.014782
5203,Aquisição de mobiliário em aço inox,Outras compras,-0.015227
5903,Fornecimento de equipamentos de refrigeração para uso nas dependências da CODESP.,Outras compras,-0.017541
92,Contratação de serviços de manutenção preventiva e corretiva dos sistemas de ar condicionado instalados nos diversos edifícios que compõem o complexo do Tribunal de Contas da União.,Outras compras,-0.018673
4918,Bolsa coletora para nefrostomia,Outras compras,-0.040029
600,Sistema de Aeronave Remotamente Pilotada (RPAS),Tecnologia da Informação,-0.048384
210,"Prestação de serviços continuados de operação de elevadores, pelo período de doze meses.",Outras compras,-0.063250
1718,Aquisição de aparelhos de ar condicionado e cortina de ar.,Outras compras,-0.094676
